In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from torchvision import models, datasets, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, auc, roc_curve
import h5py
from tqdm import tqdm
from PIL import Image
import io
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pickle

device = 'cuda' if torch.cuda.is_available else 'cpu'

import warnings
warnings.filterwarnings('ignore')

In [ ]:
test_data = h5py.File("/kaggle/input/isic-2024-challenge/test-image.hdf5", 'r')

In [ ]:
model = pickle.load(open('/kaggle/input/resnet_18_1/pytorch/default/1/model_saved3 (1).pkl', 'rb'))

In [ ]:
class dataset(Dataset):
    def __init__(self, li, transform, train):
        self.li = li
        self.transform = transform
        self.train = train
        
    def __len__(self):
        return len(self.li)
    
    def __getitem__(self, idx):
        data = self.li[idx][0]
        img = np.array(Image.open(io.BytesIO(data[()])))
        
        if self.train:
            label = self.li[idx][1]
            img = self.transform(image = img)["image"]
            return {
                "image" : img,
                "label" : label
            }
        else:
            img = self.transform(image = img)["image"]
            return{
                "image": img
            }

In [ ]:
test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [ ]:
test_li = []
ids = []

for i, data in test_data.items():
    ids.append(i)
    #img = Image.open(io.BytesIO(data[()]))
    test_li.append([data])
    del i, data
ids = np.array(ids)

In [ ]:
data_test = dataset(test_li, test_transform, False)

In [ ]:
test_load = DataLoader(data_test, batch_size=16, num_workers=4)

In [ ]:
model.eval()
pred = []
import gc
gc.collect()
with torch.inference_mode():
    for data in tqdm(test_load):
        X_test = data['image'].to(device)
        y_test_pred = nn.functional.sigmoid(model(X_test))
        pred.append(y_test_pred)
        del data, X_test, y_test_pred
        
pred = torch.cat(pred, dim = 0).cpu().numpy().reshape(-1)
pred

In [ ]:
subs = {
    "isic_id": ids,
    "target": pred
}

df = pd.DataFrame(subs)
df

In [ ]:
df.to_csv("submission.csv", index=False)